# ETL Pipeline using Python
## Extract, Transform and Load (CSV, Excel, API → MySQL Data Warehouse)

This notebook demonstrates a complete ETL pipeline:
- Extract data from CSV, Excel, and API
- Transform and clean the data
- Load the data into a MySQL Data Warehouse using XAMPP

### The librabries am going to use

Requests downloads a file from the web.
Pandas reads that file and organizes it into a table.
SQLAlchemy prepares a connection to your data_warehouse.
PyMySQL carries the data through that connection and saves it into your SQL tables.


In [3]:
!pip install pymysql

## 1. Required Libraries

In [4]:
import pandas as pd
import requests
from sqlalchemy import create_engine
import pymysql


## 2.Database Configuration

In [14]:
# MySQL Database Configuration (XAMPP)
DB_USER = "root"
DB_PASSWORD = ""   
DB_HOST = "localhost"
DB_PORT = "3306"
DB_NAME = "data_warehouse"

# Create MySQL connection engine
engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

print("Connected to MySQL Data Warehouse")


Connected to MySQL Data Warehouse


### 3.Extract
### Extract Data from CSV

In [8]:
# Extract data from CSV file
csv_df = pd.read_csv("sales_data.csv")
print("CSV Data Shape:", csv_df.shape)

csv_df.head()


CSV Data Shape: (1000, 14)


,Product_ID,Sale_Date,Sales_Rep,Region,Sales_Amount,Quantity_Sold,Product_Category,Unit_Cost,Unit_Price,Customer_Type,Discount,Payment_Method,Sales_Channel,Region_and_Sales_Rep
0,1052,2023-02-03,Bob,North,5053.97,18,Furniture,152.75,267.22,Returning,0.09,Cash,Online,North-Bob
1,1093,2023-04-21,Bob,West,4384.02,17,Furniture,3816.39,4209.44,Returning,0.11,Cash,Retail,West-Bob
2,1015,2023-09-21,David,South,4631.23,30,Food,261.56,371.40,Returning,0.20,Bank Transfer,Retail,South-David
3,1072,2023-08-24,Bob,South,2167.94,39,Clothing,4330.03,4467.75,New,0.02,Credit Card,Retail,South-Bob
4,1061,2023-03-24,Charlie,East,3750.20,13,Electronics,637.37,692.71,New,0.08,Credit Card,Online,East-Charlie


### Extract Data from Excel

In [9]:
# Extract data from Excel file
excel_df = pd.read_excel("customers.xlsx")
print("Excel Data Shape:", excel_df.shape)

excel_df.head()


Excel Data Shape: (10, 6)


,CustomerID,CustomerName,Gender,Age,Country,Email
0,1,John Doe,Male,34,USA,john@example.com
1,2,Jane Smith,Female,28,UK,jane@example.com
2,3,Michael Brown,Male,45,Canada,michael@example.com
3,4,Emily Davis,Female,31,Australia,emily@example.com
4,5,David Wilson,Male,39,Kenya,david@example.com


### Extract Data from API

In [10]:
# Extract data from API
api_url = "https://jsonplaceholder.typicode.com/users"
response = requests.get(api_url)

api_df = pd.json_normalize(response.json())
print("API Data Shape:", api_df.shape)

api_df.head()


API Data Shape: (10, 15)


,id,name,username,email,phone,website,address.street,address.suite,address.city,address.zipcode,address.geo.lat,address.geo.lng,company.name,company.catchPhrase,company.bs
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.3159,81.1496,Romaguera-Crona,Multi-layered client-server neural-net,harness real-time e-markets
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.9509,-34.4618,Deckow-Crist,Proactive didactic contingency,synergize scalable supply-chains
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info,Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.6102,-47.0653,Romaguera-Jacobson,Face to face bifurcated interface,e-enable strategic applications
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz,Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.4572,-164.2990,Robel-Corkery,Multi-tiered zero tolerance productivity,transition cutting-edge web services
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info,Skiles Walks,Suite 351,Roscoeview,33263,-31.8129,62.5342,Keebler LLC,User-centric fault-tolerant solution,revolutionize end-to-end systems


## 4.Transfrom

In [11]:
def transform_data(df):
    # Standardize column names
    df.columns = (
        df.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
    )
    
    # Remove duplicate records
    df = df.drop_duplicates()
    
    # Handle missing values
    df = df.fillna("Unknown")
    
    return df



#### Apply Transformations

In [12]:
csv_df = transform_data(csv_df)
excel_df = transform_data(excel_df)
api_df = transform_data(api_df)

print("Data transformation completed")


Data transformation completed


## 5.Load Data into Data Warehouse

In [13]:
# Load transformed data into MySQL Data Warehouse
csv_df.to_sql("sales_fact", con=engine, if_exists="replace", index=False)
excel_df.to_sql("customers_dim", con=engine, if_exists="replace", index=False)
api_df.to_sql("users_api_dim", con=engine, if_exists="replace", index=False)

print("Data successfully loaded into the Data Warehouse")


Data successfully loaded into the Data Warehouse


## ETL Pipeline Completed Successfully

The data transformation and loading process is finished. The following tables have been successfully schema-mapped and loaded into the database.

###  Tables Created in MySQL (`data_warehouse`)
* **`sales_fact`**: Contains all transactional data (Fact table).
* **`customers_dim`**: Contains detailed customer profiles (Dimension table).
* **`users_api_dim`**: Contains external user data fetched via API.

---

###  How to Verify the Data

#### Option 1: Using XAMPP (Graphical Interface)
1. Open the **XAMPP Control Panel** and ensure **MySQL** is running.
2. Click the **Admin** button next to MySQL to open **phpMyAdmin**.
3. Select the **`data_warehouse`** database from the left sidebar.
4. Click on any table name (e.g., `sales_fact`) to browse the rows.

#### Option 2: Using SQL Queries
Run the following commands in the **SQL** tab of phpMyAdmin to verify the record counts and contents:

```sql
-- Select the database
USE data_warehouse;

-- Check record counts for all tables
SELECT 'sales_fact' AS table_name, COUNT(*) AS total_rows FROM sales_fact
UNION
SELECT 'customers_dim', COUNT(*) FROM customers_dim
UNION
SELECT 'users_api_dim', COUNT(*) FROM users_api_dim;

-- Preview the first 5 rows of Sales data
SELECT * FROM sales_fact LIMIT 5;